In [ ]:
import pandas as pd
from datetime import datetime
#from data_loader import fetch_monthly_close  # or wherever you defined this

def verify_monthly_returns(
    prices: pd.Series
) -> pd.DataFrame:
    """
    Construct a table showing month-end prices and returns for verification.

    Args:
        prices (pd.Series): Month-end closing prices indexed by date.

    Returns:
        pd.DataFrame: Columns:
            - price: Month-end closing price
            - pct_change: pandas pct_change() result
            - manual_return: (price_t / price_{t-1}) - 1
    """
    df = prices.to_frame(name="price")
    df["pct_change"]    = df["price"].pct_change()
    df["manual_return"] = df["price"] / df["price"].shift(1) - 1
    return df.dropna()

# Usage example:

# Define exact window
#start_date = "2019-04-30"
#end_date   = "2024-03-31"

# Fetch month-end prices
#prices = fetch_monthly_close("PCTY", start_date=start_date, end_date=end_date)

# Build verification table
#df_verify = verify_monthly_returns(prices)

# Display to inspect
#print(df_verify)

In [ ]:
import pandas as pd
from datetime import datetime

def verify_excess_return(
    etf_prices: pd.Series,
    market_prices: pd.Series
) -> pd.DataFrame:
    """
    Create a table showing ETF returns, market returns, and excess returns.

    Args:
        etf_prices (pd.Series): Month-end prices of the ETF.
        market_prices (pd.Series): Month-end prices of the market index.

    Returns:
        pd.DataFrame: Columns:
            - etf_price
            - market_price
            - etf_return
            - market_return
            - excess_return (etf - market)
    """
    etf_df = etf_prices.to_frame(name="etf_price")
    market_df = market_prices.to_frame(name="market_price")

    df = etf_df.join(market_df, how="inner")
    df["etf_return"] = df["etf_price"].pct_change()
    df["market_return"] = df["market_price"].pct_change()
    df["excess_return"] = df["etf_return"] - df["market_return"]

    return df.dropna()

# Usage example:

# Define exact window
#start_date = "2019-04-30"
#end_date   = "2024-03-31"

# Fetch month-end prices
#prices = fetch_monthly_close("PCTY", start_date=start_date, end_date=end_date)

# Build verification table
#df_verify = verify_monthly_returns(prices)

# Display to inspect
#print(df_verify)

In [ ]:
import pandas as pd
from datetime import datetime
from typing import Optional

def preview_price_history(
    ticker: str,
    years_back: int = 5,
    end_date: Optional[Union[str, datetime]] = None
) -> pd.Series:
    """
    Fetches and displays a preview of the monthly price history for a given ticker.

    Args:
        ticker (str): The ticker symbol to fetch (e.g. "PCTY").
        years_back (int): How many years of history to pull back from today.
        end_date (str or datetime, optional): Optional end date (default = today).

    Returns:
        pd.Series: Monthly close price series.
    """
    start = pd.Timestamp.today() - pd.DateOffset(years=years_back)
    prices = fetch_monthly_close(ticker, start_date=start, end_date=end_date)

    print(f"=== Price Preview for {ticker} ===")
    print(prices.head())
    print(prices.tail())
    print(f"Fetched {len(prices)} months: {prices.index.min().date()} → {prices.index.max().date()}\n")

    return prices

# Usage example:
#preview_price_history("PCTY")

In [ ]:
import pandas as pd
from typing import Union, Optional
from datetime import datetime

def debug_single_factor_beta(
    stock_returns: pd.Series,
    factor_returns: pd.Series,
    label: str = "value"
) -> None:
    """
    Diagnostic function to verify manual beta/R² calculation between a stock and one factor.

    Args:
        stock_returns (pd.Series): Aligned stock return series (monthly).
        factor_returns (pd.Series): Aligned factor return series (monthly).
        label (str): Name of the factor (for print output).
    """
    common_idx = stock_returns.index.intersection(factor_returns.index)
    s = stock_returns.loc[common_idx]
    f = factor_returns.loc[common_idx]

    print(f"\n=== Debug: Manual Beta Calculation for {label.title()} ===")
    print("Length of stock_returns:", len(stock_returns))
    print("Length of factor_returns:", len(factor_returns))
    print("Length after intersection:", len(common_idx))

    cov = s.cov(f)
    var = f.var()
    beta = cov / var
    r_squared = s.corr(f) ** 2

    print(f"Covariance:     {cov:.6f}")
    print(f"Variance:       {var:.6f}")
    print(f"Manual Beta:    {beta:.4f}")
    print(f"Manual R²:      {r_squared:.4f}")

# Usage example:

#start = "2019-04-30"
#end = "2024-03-31"

# Required stock returns
#stock_returns = calc_monthly_returns(fetch_monthly_close("PCTY", start, end))

# One efficient call to fetch excess return (IWD - SPY)
#value_excess = fetch_excess_return("IWD", "SPY", start, end)

# Debug
#debug_single_factor_beta(stock_returns, value_excess, label="value")

In [ ]:
def debug_beta_diagnostics(
    stock_ticker: str,
    market_ticker: str,
    start_date: str,
    end_date: str
) -> None:
    """
    Runs detailed diagnostics for beta calculation using both manual (cov/var)
    and regression-based methods. Matches Excel-style variance/covariance logic.

    Args:
        stock_ticker (str): The stock symbol to analyze (e.g. "PCTY").
        market_ticker (str): The benchmark symbol (e.g. "SPY").
        start_date (str): Start date of analysis window (YYYY-MM-DD).
        end_date (str): End date of analysis window (YYYY-MM-DD).
    """
    # Fetch prices and returns
    stock_prices  = fetch_monthly_close(stock_ticker,  start_date=start_date, end_date=end_date)
    market_prices = fetch_monthly_close(market_ticker, start_date=start_date, end_date=end_date)

    stock_ret  = calc_monthly_returns(stock_prices)
    market_ret = calc_monthly_returns(market_prices)

    df_ret = pd.DataFrame({
        "stock":  stock_ret,
        "market": market_ret
    }).dropna()

    # Manual β from cov/var (sample)
    cov_sample = df_ret["stock"].cov(df_ret["market"])
    var_sample = df_ret["market"].var()
    beta_manual = cov_sample / var_sample

    # Regression-based β and diagnostics
    risk_metrics = compute_regression_metrics(df_ret)
    corr = df_ret["stock"].corr(df_ret["market"])
    r_squared = risk_metrics["r_squared"]

    # Print diagnostics
    print(f"Window: {start_date} → {end_date}")
    print(f"Observations (n):            {len(df_ret)}")
    print(f"Sample Covariance (COV.S):   {cov_sample:.6f}")
    print(f"Sample Variance (VAR.S):     {var_sample:.6f}")
    print(f"Manual β (cov/var):          {beta_manual:.4f}")
    print(f"Reported β (from regression): {risk_metrics['beta']:.4f}")
    print(f"Correlation:                 {corr:.4f}")
    print(f"R² from regression:          {r_squared:.4f}")

#Usage example
#debug_beta_diagnostics("PCTY", "SPY", "2019-05-31", "2024-03-31")

In [ ]:
def debug_single_factor_excess_beta(
    stock_ticker: str,
    factor_etf: str,
    market_ticker: str = "SPY",
    start_date: str = "2019-04-30",
    end_date: str = "2024-03-31"
) -> None:
    """
    Computes raw beta of a stock versus a factor ETF (excess return over market).
    Verifies using manual cov/var and reports components.

    Args:
        stock_ticker (str): Stock ticker to analyze.
        factor_etf (str): ETF representing the factor (e.g., "IWD" for value).
        market_ticker (str): Market ETF to subtract from factor (default "SPY").
        start_date (str): Start date of the window (YYYY-MM-DD).
        end_date (str): End date of the window (YYYY-MM-DD).
    """
    # Fetch prices and calculate returns
    stock_prices  = fetch_monthly_close(stock_ticker, start_date, end_date)
    factor_prices = fetch_monthly_close(factor_etf,    start_date, end_date)
    market_prices = fetch_monthly_close(market_ticker, start_date, end_date)

    stock_returns = calc_monthly_returns(stock_prices)
    factor_returns = calc_monthly_returns(factor_prices)
    market_returns = calc_monthly_returns(market_prices)
    excess_returns = factor_returns - market_returns

    # Align dates
    common_idx = stock_returns.index.intersection(excess_returns.index)
    s = stock_returns.loc[common_idx]
    x = excess_returns.loc[common_idx]

    # Compute raw beta components
    cov = s.cov(x)
    var = x.var()
    beta = cov / var

    # Output
    print(f"n_obs: {len(common_idx)}")
    print(f"cov:   {cov:.6f}")
    print(f"var:   {var:.6f}")
    print(f"beta:  {beta:.4f}")

#Usage example
#debug_single_factor_excess_beta("PCTY", "IWD", "SPY", "2019-04-30", "2024-03-31")